# Keras Quickstart for the AMEX competition part 2: Inference

This notebook uses the trained models of [Keras Quickstart 1: Training]() and computes the test predictions.


In [ ]:
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt
import random
import datetime
from matplotlib.ticker import MaxNLocator
import os

import tensorflow as tf


# Inference

We perform the same steps as in the training pipeline:
- Read the test data
- Keep only the most recent statement of every customer
- One-hot encode the categorical features
- Impute missing values
- Scale the data with the previously saved scaler (for every fold)

Then we predict the target with every model that we have saved before and submit the median of the predictions. We could take the mean instead of the median - I do not yet know whether that makes a difference.


In [ ]:
%%time
model_dir = '../input/amex-keras-quickstart-1-training'
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
with open(os.path.join(model_dir, "ohe.pickle"), 'rb') as f: ohe = pickle.load(f)
#with open('../input/pickled-shrunken-training-data/test_data.pkl', 'rb') as f: test = pickle.load(f)
test = pd.read_feather('../input/parquet-files-amexdefault-prediction/test_data.ftr')
num_statements = (test
          .groupby('customer_ID')
          .size()
          .sort_index())

# Keep only the most recent statement of every customer
test = (test
         .groupby('customer_ID')
         .tail(1)
         .set_index('customer_ID', drop=True)
         .sort_index()
         .drop(['S_2'], axis='columns'))
test['num_statements'] = num_statements

# One-hot encode the categorical features
not_cat_features = [f for f in test.columns if f not in cat_features]
test[cat_features] = test[cat_features].astype(object)
test = pd.concat([test[not_cat_features],
                   pd.DataFrame(ohe.transform(test[cat_features]), index=test.index).rename(columns=str)], axis=1)

# Impute missing values
test.fillna(value=0, inplace=True)

fold = 0
y_pred_list = []
try:
    while True: # loop over all folds
        # Load the scaler and the model
        with open(os.path.join(model_dir, f"scaler_{fold}.pickle"), 'rb') as f: scaler = pickle.load(f)
        model = tf.keras.models.load_model(os.path.join(model_dir, f"model_{fold}"))

        # Scale and predict
        y_pred_list.append(model.predict(scaler.transform(test), batch_size=1024*1024, verbose=0).ravel())
        with np.printoptions(linewidth=150, precision=2, suppress=True):
            print(fold, y_pred_list[-1])

        fold += 1
except FileNotFoundError:
    pass

# Ensemble the predictions of all folds
sub = pd.DataFrame({'customer_ID': test.index,
                    'prediction': np.median(y_pred_list, axis=0)})


In [ ]:
sub.to_csv('submission.csv', index=False)
sub

As a plausibility test, we plot a histogram of the predictions. The majority of the predictions should be near 0 (because the classes are imbalanced).

In [ ]:
plt.figure(figsize=(16, 5))
plt.hist(sub.prediction, bins=100, density=True)
plt.title("Plausibility check", fontsize=20)
plt.xlabel('Prediction')
plt.ylabel('Density')
plt.show()